In [19]:
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\Jay\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm.auto import tqdm

download dataset
create dataloader
build model 
train
save trained model

In [5]:
def download_mnist_datasets():
    train_data = datasets.MNIST(
    root='data',
    download=True,
    train=True,
    transform=ToTensor())
    test_data = datasets.MNIST(
    root='data',
        download=True,
        train=False,
        transform=ToTensor()
    )
    return train_data , test_data

In [6]:
train_dataset , test_dataset = download_mnist_datasets()

100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [7]:
# Create a dataloader
Batch=128

train_data_loader = DataLoader(train_dataset, batch_size=Batch)
test_data_loader = DataLoader(test_dataset, batch_size=Batch)

In [8]:
# Build model
class FedForNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=28*28,out_features=256),
            nn.ReLU(),
            nn.Linear(256,10))
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        return self.softmax(self.dense_layers(self.flatten(x)))
    

In [9]:
# Steup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [11]:
model_0 = FedForNet().to(device)
model_0

FedForNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense_layers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [24]:
def train_step(model, data_loader, loss_fn, optimizer, device):
    
    model.train()
    for X,y in data_loader:
        X , y = X.to(device) , y.to(device)
        
        y_pred = model(X)
        
        loss = loss_fn(y_pred,y)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    print(f"Loss: {loss}")

In [28]:
def test_step(model, data_loader, loss_fn, device):
    
    model.eval()
    
    with torch.inference_mode():
        
        for X,y in data_loader:
            X , y = X.to(device) , y.to(device)

            test_pred = model(X)

            test_loss = loss_fn(test_pred,y)

        
    print(f"Test Loss: {test_loss}")

In [29]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model_0.parameters(),
                            lr=0.01)

In [30]:
# Set the number of epochs
epochs = 3

for epoch in tqdm(range(epochs)):
      print(f"Epoch: {epoch} \n-----------")
      ### Training
      train_step(model=model_0,
                   data_loader=train_data_loader,
                   loss_fn=loss_fn,
                   optimizer=optimizer,
                   device=device)

      ###testing
      test_step(model=model_0,
                   data_loader=test_data_loader,
                   loss_fn=loss_fn,
                   device=device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0 
-----------
Loss: 1.5735770463943481
Test Loss: 1.4611505270004272
Epoch: 1 
-----------
Loss: 1.4871691465377808
Test Loss: 1.4611505270004272
Epoch: 2 
-----------
Loss: 1.48198401927948
Test Loss: 1.4611505270004272


In [31]:
model_0.state_dict()

OrderedDict([('dense_layers.0.weight',
              tensor([[ 0.0268,  0.0315, -0.0315,  ...,  0.0005,  0.0354,  0.0341],
                      [ 0.0325, -0.0021,  0.0209,  ..., -0.0155,  0.0031, -0.0029],
                      [ 0.0247, -0.0138,  0.0057,  ...,  0.0342,  0.0341, -0.0184],
                      ...,
                      [ 0.0318, -0.0264, -0.0246,  ..., -0.0283,  0.0346,  0.0240],
                      [-0.0050,  0.0076, -0.0201,  ..., -0.0344, -0.0085, -0.0130],
                      [-0.0182, -0.0266, -0.0290,  ..., -0.0187,  0.0352,  0.0309]],
                     device='cuda:0')),
             ('dense_layers.0.bias',
              tensor([-8.6941e-02,  1.0909e-01,  4.2577e-02, -1.2244e-02, -1.7834e-01,
                       3.7485e-01, -7.1277e-02,  1.1748e-01, -1.7788e-01, -4.0114e-02,
                      -1.2993e-01, -1.8325e-01, -1.7881e-02, -4.5086e-01, -1.2966e-01,
                      -8.2375e-02, -3.1820e-01, -6.4066e-02,  3.3100e-01,  3.0859e-01,
    

In [32]:
torch.save(model_0.state_dict(), "FedFornet.pth")
print("Model stored")

Model stored
